# Import libraries

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

# Load training dataset

In [ ]:
train_dataframe = pd.read_csv('dataset/1. istanbul/train_data.csv')

# Visualize some training data

In [ ]:
train_dataframe.head()

# Prepare training data

In [ ]:
train_x = train_dataframe.drop(['Subject ID', 'Class information', 'UPDRS'], axis=1)
train_x = train_x.as_matrix()
print(train_x.shape)

# Prepare training labels

In [ ]:
train_y = train_dataframe['Class information']
train_y = train_y.as_matrix()
print(train_y.shape)
train_y = np_utils.to_categorical(train_y)
print(train_y.shape)

# Prepare test data

In [ ]:
test_dataframe = pd.read_csv('dataset/1. istanbul/test_data.csv')
test_x = test_dataframe.drop(['Subject ID', 'Class information'], axis=1)
test_x = test_x.as_matrix()
print(test_x.shape)

# Prepare test labels

In [ ]:
test_y = test_dataframe['Class information']
test_y = test_y.as_matrix()
print(test_y.shape)
test_y = np_utils.to_categorical(test_y)
print(test_y.shape)

# Combine train and test set

In [ ]:
train_x = np.vstack([train_x, test_x])
train_y = np.vstack([train_y, test_y])
print(train_x.shape)
print(train_y.shape)

# Normalize data

In [ ]:
mins = np.min(train_x, axis=0)
maxs = np.max(train_x, axis=0)
rng = maxs - mins
train_x = 1.0 - (((1.0 - 0.0) * (maxs - train_x)) / rng)
test_x = 1.0 - (((1.0 - 0.0) * (maxs - test_x)) / rng)
pd.DataFrame(train_x).head()

# Shuffle data

In [ ]:
train_x, train_y = shuffle(train_x, train_y)
pd.DataFrame(train_y).head()

# Create model

In [ ]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/results-gridsearch-02/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
validation_split = 0.172;
def create_model(layers, optimizer):
    global date
    x = Input(shape=(train_x.shape[1],))

    y = Dense(units = layers[0], activation='relu')(x)
    y = Dropout(0.5)(y)

    y = Dense(units = layers[1], activation='relu')(y)
    y = Dropout(0.5)(y)
    
    y = Dense(units = layers[2], activation='relu')(y)
    y = Dropout(0.5)(y)
    
    if (layers[3] > 0):
        y = Dense(units = layers[3], activation='relu')(y)
        y = Dropout(0.5)(y)

    y = Dense(units = train_y.shape[1], activation='softmax')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    file = open(directory + "params.txt", "a")
    file.write("optimizer: %s, layer 1: %d, layer 2: %d, layer 3: %d, layer 4: %d" % (optimizer[0], layers[0], layers[1], layers[2], layers[3]))
    file.close()
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda x: file2.write(x + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=optimizer[1], loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [ ]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        callbacks = [tensorboard_callback, csv_logger]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, **kwargs)

# Grid search hyperparameters

In [ ]:
model = KerasClassifierTensorBoard(build_fn=create_model, epochs=6000, batch_size=20)

layers = [
    [4, 8, 16, 0],
    [4, 32, 16, 0],
    [4, 64, 8, 0],
    [4, 64, 8, 6],
    [4, 64, 32, 0],
    [6, 6, 32, 0],
    [6, 6, 32, 64],
    [6, 6, 32, 16],
    [6, 6, 32, 8],
    [6, 6, 64, 0]
]
optimizers = [
    ['Adam(lr=0.0001, decay=0)', keras.optimizers.Adam(lr=0.0001, decay=0)],
    ['Adam(lr=0.0001, decay=0.00001)', keras.optimizers.Adam(lr=0.0001, decay=0.00001)],
    ['Adam(lr=0.0001, decay=0.000001)', keras.optimizers.Adam(lr=0.0001, decay=0.000001)],
    ['Adam(lr=0.00001, decay=0.000001)', keras.optimizers.Adam(lr=0.00001, decay=0.000001)],
    ['RMSprop(lr=0.0001, decay=0)', keras.optimizers.RMSprop(lr=0.0001, decay=0)],
    ['RMSprop(lr=0.0001, decay=0.00001)', keras.optimizers.RMSprop(lr=0.0001, decay=0.00001)],
    ['RMSprop(lr=0.0001, decay=0.000001)', keras.optimizers.RMSprop(lr=0.0001, decay=0.000001)],
    ['RMSprop(lr=0.00001, decay=0.000001)', keras.optimizers.RMSprop(lr=0.00001, decay=0.000001)]
]
param_grid = dict(
    layers=layers, 
    optimizer = optimizers
)

grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_split=validation_split))
grid_result = grid.fit(train_x, train_y)

# Print results

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean %f, Std %f with: %r" % (mean, stdev, param))

# The End